## Step 1 - Flask API

Use Flask to design an API for your dataset and to serve the HTML and JavaScript required for your dashboard page. Note: We recommend using the sqlite database file and SQLAlchemy inside of your Flask application code, but you are permitted to read the CSV data directly into Pandas DataFrames for this assignment. You will still need to output the data as JSON in the format specified in the routes below.

* First, create a template called `index.html` for your dashboard landing page. Use the Bootstrap grid system to create the structure of the dashboard page.
* Next, create the following routes for your api.

```python
@app.route("/")
    """Return the dashboard homepage."""
```
```python
@app.route('/names')
    """List of sample names.

    Returns a list of sample names in the format
    [
        "BB_940",
        "BB_941",
        "BB_943",
        "BB_944",
        "BB_945",
        "BB_946",
        "BB_947",
        ...
    ]

    """
```
```python
@app.route('/otu')
    """List of OTU descriptions.

    Returns a list of OTU descriptions in the following format

    [
        "Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus",
        "Archaea;Euryarchaeota;Halobacteria;Halobacteriales;Halobacteriaceae;Halococcus",
        "Bacteria",
        "Bacteria",
        "Bacteria",
        ...
    ]
    """
```
```python
@app.route('/metadata/<sample>')
    """MetaData for a given sample.

    Args: Sample in the format: `BB_940`

    Returns a json dictionary of sample metadata in the format

    {
        AGE: 24,
        BBTYPE: "I",
        ETHNICITY: "Caucasian",
        GENDER: "F",
        LOCATION: "Beaufort/NC",
        SAMPLEID: 940
    }
    """
```
```python
@app.route('/wfreq/<sample>')
    """Weekly Washing Frequency as a number.

    Args: Sample in the format: `BB_940`

    Returns an integer value for the weekly washing frequency `WFREQ`
    """
```
```python
@app.route('/samples/<sample>')
    """OTU IDs and Sample Values for a given sample.

    Sort your Pandas DataFrame (OTU ID and Sample Value)
    in Descending Order by Sample Value

    Return a list of dictionaries containing sorted lists  for `otu_ids`
    and `sample_values`

    [
        {
            otu_ids: [
                1166,
                2858,
                481,
                ...
            ],
            sample_values: [
                163,
                126,
                113,
                ...
            ]
        }
    ]
    """
```


In [33]:
import pandas as pd
import json
import pprint
import csv
import pandas as pd

from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)


In [47]:
metadata = "C:\\Users\\ushuel00\\Downloads\\UCI\\UCIRV201801DATA3-Class-Repository-DATA\\02-Homework\\15-Interactive-Visualizations-and-Dashboards\\Instructions\\DataSets\\Belly_Button_Biodiversity_Metadata.csv"
otu_id = "C:\\Users\\ushuel00\\Downloads\\UCI\\UCIRV201801DATA3-Class-Repository-DATA\\02-Homework\\15-Interactive-Visualizations-and-Dashboards\\Instructions\\DataSets\\belly_button_biodiversity_otu_id.csv"
metadata_columns = "C:\\Users\\ushuel00\\Downloads\\UCI\\UCIRV201801DATA3-Class-Repository-DATA\\02-Homework\\15-Interactive-Visualizations-and-Dashboards\\Instructions\\DataSets\\metadata_columns.csv"
sample = "C:\\Users\\ushuel00\\Downloads\\UCI\\UCIRV201801DATA3-Class-Repository-DATA\\02-Homework\\15-Interactive-Visualizations-and-Dashboards\\Instructions\\DataSets\\belly_button_biodiversity_samples.csv"


In [48]:
metadata = pd.read_csv(metadata)
otu_id = pd.read_csv(otu_id)
metadata_columns = pd.read_csv(metadata_columns)
sample = pd.read_csv(sample)

In [31]:
metadata.head()

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


def make_record(row):
    return {"personalinfo": {
                "AGE": row["AGE"],
                "BBTYPE": row["BBTYPE"],
                "ETHNICITY": row["ETHNICITY"],
                "GENDER": row["GENDER"],
                "LOCATION": row["LOCATION"],
                "SAMPLEID": row["SAMPLEID"]}
           }
           

with open('C:\\Users\\ushuel00\\Downloads\\UCI\\UCIRV201801DATA3-Class-Repository-DATA\\02-Homework\\15-Interactive-Visualizations-and-Dashboards\\Instructions\\DataSets\\Belly_Button_Biodiversity_Metadata.csv', 'r', newline='') as csvfile:
    reader = csv.DictReader(metadata, delimiter='\t')
    with open('metadata_json', 'w') as jsonfile:
        out = json.dumps([make_record(row) for row in reader])
        jsonfile.write(out)

# show result
with open('metadata_json.json', 'r') as jsonfile:
    print('results:')
    print(json.dumps(json.load(jsonfile), indent=4))

In [49]:
#Read CSV File
def read_csv(metadata, json_file, format):
    csv_rows = []
    with open(metadata) as csvfile:
        reader = csv.DictReader(csvfile)
        title = reader.fieldnames
        for row in reader:
            csv_rows.extend([{title[i]:row[title[i]] for i in range(len(title))}])
        write_json(csv_rows, metadata_json, format)

#Convert csv data into json and write it
def write_json(data, metadata_json, format):
    with open(metadata_json, "w") as f:
        if format == "pretty":
            f.write(json.dumps(data, sort_keys=False, indent=4, separators=(',', ': '),encoding="utf-8",ensure_ascii=False))
        else:
            f.write(json.dumps(data))

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(metadata_json)

'[{"SAMPLEID":940,"EVENT":"BellyButtonsScienceOnline","ETHNICITY":"Caucasian","GENDER":"F","AGE":24.0,"WFREQ":2.0,"BBTYPE":"I","LOCATION":"Beaufort\\/NC","COUNTRY012":"usa","ZIP012":"22306","COUNTRY1319":"usa","ZIP1319":"22306","DOG":"no","CAT":"no","IMPSURFACE013":8852.0,"NPP013":37.17222214,"MMAXTEMP013":54.5,"PFC013":null,"IMPSURFACE1319":"1","NPP1319":null,"MMAXTEMP1319":33.99000168,"PFC1319":25.5},{"SAMPLEID":941,"EVENT":null,"ETHNICITY":"Caucasian\\/Midleastern","GENDER":"F","AGE":34.0,"WFREQ":1.0,"BBTYPE":"I","LOCATION":"Chicago\\/IL","COUNTRY012":null,"ZIP012":null,"COUNTRY1319":null,"ZIP1319":null,"DOG":null,"CAT":null,"IMPSURFACE013":null,"NPP013":null,"MMAXTEMP013":null,"PFC013":null,"IMPSURFACE1319":null,"NPP1319":null,"MMAXTEMP1319":null,"PFC1319":null},{"SAMPLEID":943,"EVENT":"BellyButtonsScienceOnline","ETHNICITY":"Caucasian","GENDER":"F","AGE":49.0,"WFREQ":1.0,"BBTYPE":"I","LOCATION":"Omaha\\/NE","COUNTRY012":null,"ZIP012":null,"COUNTRY1319":null,"ZIP1319":null,"DOG":nu

In [ ]:
    # Format the data for Plotly
    plot_trace = {
            "x": df["emoji_id"].values.tolist(),
            "y": df["score"].values.tolist(),
            "type": "bar"
    }
    return jsonify(plot_trace)

In [7]:
otu_id.head()

,otu_id,lowest_taxonomic_unit_found
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...


In [8]:
metadata_columns.head()

,COLUMN,TYPE,DESCRIPTION
0,SAMPLEID,INTEGER,sample_id
1,EVENT,TEXT,Collection event
2,ETHNICITY,TEXT,Ethnicity
3,GENDER,TEXT,Gender (male/female)
4,AGE,INTEGER,Age (in years)


sample.head()

In [18]:
metadata_json = metadata.to_json(orient='records')
pprint.pprint(metadata_json, depth = 3)

'[{"SAMPLEID":940,"EVENT":"BellyButtonsScienceOnline","ETHNICITY":"Caucasian","GENDER":"F","AGE":24.0,"WFREQ":2.0,"BBTYPE":"I","LOCATION":"Beaufort\\/NC","COUNTRY012":"usa","ZIP012":"22306","COUNTRY1319":"usa","ZIP1319":"22306","DOG":"no","CAT":"no","IMPSURFACE013":8852.0,"NPP013":37.17222214,"MMAXTEMP013":54.5,"PFC013":null,"IMPSURFACE1319":"1","NPP1319":null,"MMAXTEMP1319":33.99000168,"PFC1319":25.5},{"SAMPLEID":941,"EVENT":null,"ETHNICITY":"Caucasian\\/Midleastern","GENDER":"F","AGE":34.0,"WFREQ":1.0,"BBTYPE":"I","LOCATION":"Chicago\\/IL","COUNTRY012":null,"ZIP012":null,"COUNTRY1319":null,"ZIP1319":null,"DOG":null,"CAT":null,"IMPSURFACE013":null,"NPP013":null,"MMAXTEMP013":null,"PFC013":null,"IMPSURFACE1319":null,"NPP1319":null,"MMAXTEMP1319":null,"PFC1319":null},{"SAMPLEID":943,"EVENT":"BellyButtonsScienceOnline","ETHNICITY":"Caucasian","GENDER":"F","AGE":49.0,"WFREQ":1.0,"BBTYPE":"I","LOCATION":"Omaha\\/NE","COUNTRY012":null,"ZIP012":null,"COUNTRY1319":null,"ZIP1319":null,"DOG":nu

In [20]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(metadata_json)

'[{"SAMPLEID":940,"EVENT":"BellyButtonsScienceOnline","ETHNICITY":"Caucasian","GENDER":"F","AGE":24.0,"WFREQ":2.0,"BBTYPE":"I","LOCATION":"Beaufort\\/NC","COUNTRY012":"usa","ZIP012":"22306","COUNTRY1319":"usa","ZIP1319":"22306","DOG":"no","CAT":"no","IMPSURFACE013":8852.0,"NPP013":37.17222214,"MMAXTEMP013":54.5,"PFC013":null,"IMPSURFACE1319":"1","NPP1319":null,"MMAXTEMP1319":33.99000168,"PFC1319":25.5},{"SAMPLEID":941,"EVENT":null,"ETHNICITY":"Caucasian\\/Midleastern","GENDER":"F","AGE":34.0,"WFREQ":1.0,"BBTYPE":"I","LOCATION":"Chicago\\/IL","COUNTRY012":null,"ZIP012":null,"COUNTRY1319":null,"ZIP1319":null,"DOG":null,"CAT":null,"IMPSURFACE013":null,"NPP013":null,"MMAXTEMP013":null,"PFC013":null,"IMPSURFACE1319":null,"NPP1319":null,"MMAXTEMP1319":null,"PFC1319":null},{"SAMPLEID":943,"EVENT":"BellyButtonsScienceOnline","ETHNICITY":"Caucasian","GENDER":"F","AGE":49.0,"WFREQ":1.0,"BBTYPE":"I","LOCATION":"Omaha\\/NE","COUNTRY012":null,"ZIP012":null,"COUNTRY1319":null,"ZIP1319":null,"DOG":nu


## Step 2 - Plotly.js

Use Plotly.js to build interactive charts for your dashboard.

* Use the route `/names` to populate a dropdown select element with the list of sample names.

  * Use `document.getElementById`, `document.createElement` and `append` to populate the create option elements and append them to the dropdown selector.

  * Use the following HTML tag for the dropdown selector

  ```html
  <select id="selDataset" onchange="optionChanged(this.value)"></select>
  ```
  * Create a function called `optionChanged` to handle the change event when a new sample is selected (i.e. fetch data for the newly selected sample).

  ![dropdown](Images/dropdown.png)

* Create a PIE chart that uses data from your routes `/samples/<sample>` and `/otu` to display the top 10 samples.

  * Use the Sample Value as the values for the PIE chart

  * Use the OTU ID as the labels for the pie chart

  * Use the OTU Description as the hovertext for the chart

  * Use `Plotly.restyle` to update the chart whenever a new sample is selected

  ![PIE Chart](Images/pie_chart.png)

* Create a Bubble Chart that uses data from your routes `/samples/<sample>` and `/otu` to plot the __Sample Value__ vs the __OTU ID__ for the selected sample.

  * Use the OTU IDs for the x values

  * Use the Sample Values for the y values

  * Use the Sample Values for the marker size

  * Use the OTU IDs for the marker colors

  * Use the OTU Description Data for the text values

  * Use `Plotly.restyle` to update the chart whenever a new sample is selected

  ![Bubble Chart](Images/bubble_chart.png)

* Display the sample metadata from the route `/metadata/<sample>`

  * Display each key/value pair from the metadata JSON object somewhere on the page

  * Update the metadata for each sample that is selected

* You are welcome to create any layout that you would like for your dashboard. An example dashboard page might look something like the following.

![Example Dashboard Page](Images/dashboard_part1.png)
![Example Dashboard Page](Images/dashboard_part2.png)